# Docs
https://developer.sportradar.com/ice-hockey/reference/nhl-game-play-by-play

### event-types

https://developer.sportradar.com/ice-hockey/reference/nhl-faqs#event-types

- gamesetup
- substitution
- faceoff
- hit
- shotsaved
- giveaway
- shotmissed
- stoppage
- penalty
- goal
- takeaway
- 

### shot_type
- tip-in , tip
- wrist
- backhand
- deflected
- slap
- snap
- bat
- wrap-around
- cradle
- poke
- between_legs

In [1]:
import json;

In [4]:
%run lib.ipynb

In [5]:
seconds_in_period = timeToSeconds('20:00')
seconds_in_overtime = timeToSeconds('20:00')

## REPORT

In [52]:
gameFileId = '8f36d70b-a463-452e-9f02-65dd5d37d2c9'

with open( f"./output/games/game_{gameFileId}.json") as f:
    data = json.load(f)

print(data['id'])
gameo = DictObj(data)


8f36d70b-a463-452e-9f02-65dd5d37d2c9


In [72]:

home_counter = []
away_counter = []

home_reference = gameo.home.reference
away_reference = gameo.away.reference

report = {
    'teams':{
        'home':{
            'reference': gameo.home.reference,
            'name': gameo.home.name,
            'market': gameo.home.market,
        },
        'away':{
            'reference': gameo.away.reference,
            'name': gameo.away.name,
            'market': gameo.away.market,
        }
    },
    'scoring':{
        'total':{
            'home': gameo.home.points,
            'away': gameo.away.points,
        },
        'periods':[]
    },
    'shoots':{}

}


for period in gameo.periods:
    
    print(period.sequence)
    
    scoringObj = {
        'home': period.scoring.home.points,
        'away': period.scoring.away.points,
        'period_sequence':  period.sequence,
        'period_number':  period.number,
        'period_type':  period.type
    }
    report['scoring']['periods'].append(scoringObj)
    
    for event in period.events:
          
        
        if( event.event_type == "shotmissed" or  
            event.event_type == "shotsaved" or 
            event.event_type == "goal" ):

            current_len = 0

            if(event.attribution.reference == home_reference):
                current_len = len(home_counter) + 1
            else:
                current_len = len(away_counter) + 1
               
            
            secondsPeriodByType = seconds_in_period
            
            if( period.type == 'overtime'):
                 secondsPeriodByType = seconds_in_overtime
                    
            timeInPeriod = secondsPeriodByType - timeToSeconds(event.clock)
            
            timeInGame = 0
            
            if( period.sequence == 1):
                timeInGame = timeInPeriod
            elif ( period.sequence == 2 or period.sequence == 3):
                 timeInGame = ((period.number - 1) * secondsPeriodByType ) + timeInPeriod
            else:
                # overtime
                timeInGame = (seconds_in_period * 3 )  +  ((period.sequence - 4) * seconds_in_overtime ) + timeInPeriod
                
            obj =  {
                'clock_in_period':  event.clock,
                'time_game_seconds': timeInGame,
                'time_period_seconds':  timeInPeriod ,
                'shoot_type':  event.event_type,
                'shoot_counter_onclock': current_len,
                'period_number':  period.number,
                'period_type':  period.type,
                'period_sequence':  period.sequence,
                'attribution': 'home' if event.attribution.reference == gameo.home.reference else 'away'
            }

            if(event.attribution.reference == home_reference):
                home_counter.append(obj)
            else:
                away_counter.append(obj)


report['shoots'] = {
    'home':home_counter,
    'away':away_counter,
}
                
# print(report)

file_name = f"./output/reports/shoots-counter/shoots-counter_{gameo.id}.json"
with open(file_name, "w") as outfile:
    json.dump(report, outfile, indent=2)

1
2
3
4
5


In [17]:
# print(event.clock, event.attribution.market, event.event_type , event.details.distance)
# print(event.location.coord_x, event.location.coord_x, event.location.action_area) 

#     print(event.description)
#       'zone': event.zone,
#     print('---',)
#     print('---', )
#     print(event.description)
#     print(event.details) #shot_type, distance
#     print('---', event.attribution.id)
#     print('---', event.attribution.id)
